#### BI1 Mini-Project: UOL_Bibliography

In [1]:
import pandas as pd
import os
import glob
import numpy as np
import re

In [10]:
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "Data/uol-bibliography-data-2008-2017-raw/", "*.csv"))

In [11]:
df = pd.core.frame.DataFrame(
    {
        
    })

Empty DataFrame
Columns: []
Index: []


In [14]:
for f in csv_files:
    tmp = pd.read_csv(f)
    try:
        tmp = tmp.assign(Faculty = len(tmp)*re.findall('Fakultt ([1-9])',f))
    except:
        tmp = tmp.assign(Faculty = len(tmp)*[np.nan])
    if tmp['Fach'][0].startswith('%') == False:
        df = pd.concat([df,tmp],ignore_index=True)

                                                    Fach            Autor/in  \
0      Fakultät 5 - Mathematik und Naturwissenschafte...  Ahrenholtz, Ingrid   
1      Fakultät 5 - Mathematik und Naturwissenschafte...         Ruf, Andrea   
2                                   Rechtswissenschaften      Balitzki, Anja   
3                                   Rechtswissenschaften      Balitzki, Anja   
4                                   Rechtswissenschaften      Balitzki, Anja   
...                                                  ...                 ...   
56869                                     Stadtforschung      Siebel, Walter   
56870                                     Stadtforschung      Siebel, Walter   
56871                                     Stadtforschung      Siebel, Walter   
56872                                     Stadtforschung       Wehrheim, Jan   
56873                                     Stadtforschung       Wehrheim, Jan   

                                       

In [15]:
df[['lastname', 'firstname']] = df['Autor/in'].str.split(', ', 1, expand=True)
df[['title', 'pages']] = df['Titel'].str.split(' \(', 1, expand=True)
df[['pages']] = df.pages.str.extract('(\d+)')

                                                    Fach            Autor/in  \
0      Fakultät 5 - Mathematik und Naturwissenschafte...  Ahrenholtz, Ingrid   
1      Fakultät 5 - Mathematik und Naturwissenschafte...         Ruf, Andrea   
2                                   Rechtswissenschaften      Balitzki, Anja   
3                                   Rechtswissenschaften      Balitzki, Anja   
4                                   Rechtswissenschaften      Balitzki, Anja   
...                                                  ...                 ...   
56869                                     Stadtforschung      Siebel, Walter   
56870                                     Stadtforschung      Siebel, Walter   
56871                                     Stadtforschung      Siebel, Walter   
56872                                     Stadtforschung       Wehrheim, Jan   
56873                                     Stadtforschung       Wehrheim, Jan   

                                       

In [16]:
df = df.drop(['Autor/in','Titel'],axis=1)

In [17]:
df = df.rename(columns={"Fach": "institute",'Typ':'type', 'Meldetag':'date', 'Punktzahl':'points', ' ZahlOldenburgerAutoren':'num_authors', 'Jahr':'year', 'Faculty':'faculty'})

In [77]:
df.to_csv('Data/full_data.csv',index=False)

In [19]:
tmp = pd.read_json('Data/citations-db.json')
cit = pd.DataFrame(data = {'title': tmp.columns.values.tolist(), 'citations': tmp.iloc[0].values.astype(str)})
cit[['citations']] = cit.citations.str.extract('(\d+)')

,title,citations
0,,0
1,Microbial communities releasing methane in aba...,30
2,Lehrervorstellungen zum Experimentieren im Bio...,0
3,Literaturhinweise zum Themenfeld Massenmedien ...,0
4,Entwicklung und Klassifikation energiebewusste...,1
...,...,...
14448,OlVis,0
14449,Discriminative learning of relevant percepts f...,3
14450,Regional PV power prediction for improved grid...,90
14451,Recyclable gallium as catalyst precursor for a...,0


In [20]:
final = pd.merge(df,cit,on = 'title',how='left')

,institute,type,date,points,num_authors,year,faculty,lastname,firstname,title,pages,citations
0,Fakultät 5 - Mathematik und Naturwissenschafte...,Aufsatz,28.04.15,0.75,2,2014,NaN,Ahrenholtz,Ingrid,Akzeptanz und Erfolg von zusätzlichen Maßnahme...,7,0
1,Fakultät 5 - Mathematik und Naturwissenschafte...,Aufsatz,28.04.15,0.75,2,2014,NaN,Ruf,Andrea,Akzeptanz und Erfolg von zusätzlichen Maßnahme...,7,0
2,Rechtswissenschaften,Aufsatz,15.04.13,0.50,2,2012,2,Balitzki,Anja,Beware of the Dog,6,0
3,Rechtswissenschaften,Aufsatz,29.08.12,1.50,1,2012,2,Balitzki,Anja,Der kartellrechtliche Zwangslizenzeinwand bei ...,12,0
4,Rechtswissenschaften,Aufsatz,15.04.13,0.50,2,2012,2,Balitzki,Anja,Luxusgut Hund? Plädoyer für einen reflektierte...,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...
56869,Stadtforschung,Aufsatz,13.06.14,1.00,1,2013,1,Siebel,Walter,Kolonialisierung durch das Ästhetische?,4,0
56870,Stadtforschung,Monographie,13.08.13,3.00,1,2013,1,Siebel,Walter,Polarisierte Städte,351,1
56871,Stadtforschung,Aufsatz,13.06.14,1.00,1,2013,1,Siebel,Walter,Stadtkronenpolitik,6,0
56872,Stadtforschung,Aufsatz,16.08.11,1.00,1,2011,1,Wehrheim,Jan,"Kriminologie, Sicherheit und die herrschende N...",17,0


In [98]:
final.to_csv('Data/230116_full_data.csv',index=False)

In [9]:
final = pd.read_csv('Data/230116_full_data.csv')

0        False
1        False
2        False
3        False
4        False
         ...  
56871    False
56872    False
56873    False
56874    False
56875    False
Name: institute, Length: 56876, dtype: bool

In [160]:
years = pd.DataFrame(data={'year': final.year}).drop_duplicates()
years = years.sort_values('year')
years = pd.DataFrame(data={'yID': range(1,len(years)+1),'year': years.year})
years = years.reset_index(drop=True)

In [164]:
final['date'] = pd.to_datetime(final['date'])
dates = pd.DataFrame(data={'dateID': range(1,11654),'date': pd.date_range(pd.to_datetime(min(final['date'])), 
                                                                          pd.to_datetime(max(final['date']) 
                                                                                         + pd.Timedelta(1, unit='D')))})

In [148]:
departments = pd.DataFrame(data={'department': final.institute, 'faculty': final.faculty}).drop_duplicates()
departments = departments.sort_values('department')
departments = pd.DataFrame(data={'depID': range(1,len(departments)+1), 'department': departments.department, 
                                 'faculty': departments.faculty})
departments = departments.reset_index(drop=True)

In [150]:
authors = pd.DataFrame(data={'lastname': final.lastname, 'firstname': final.firstname}).drop_duplicates()
authors = authors.sort_values(['lastname','firstname'])
authors = pd.DataFrame(data={'aID': range(1,len(authors)+1), 'lastname': authors.lastname, 
                             'firstname': authors.firstname})
authors = authors.reset_index(drop=True)

In [151]:
publications = pd.DataFrame(data={'title': final.title, 'type': final.type, 'num_authors': final.num_authors,
                                 'pages': final.pages, 'citations': final.citations}).drop_duplicates()
publications = publications.sort_values('title')
publications = pd.DataFrame(data={'pID': range(1,len(publications)+1), 'title': publications.title, 
                                  'type': publications.type, 'num_authors': publications.num_authors, 
                                  'pages': publications.pages, 'citations': publications.citations})
publications = publications.reset_index(drop=True)

In [153]:
facts = pd.merge(final,departments,how='left',left_on='institute',right_on='department').drop(['institute',
                                                                                               'faculty_x',
                                                                                               'faculty_y',
                                                                                               'department'],axis=1)
facts = pd.merge(facts,dates,how='left',on='date').drop('date',axis=1)
facts = pd.merge(facts,years,how='left',on='year').drop('year',axis=1)
facts = pd.merge(facts,publications,how='left',on='title').drop(['title','type_x','num_authors_x','pages_x',
                                                                 'citations_x','type_y','num_authors_y','pages_y',
                                                                 'citations_y'],axis=1)
facts = pd.merge(facts,authors,how='left',on=['lastname','firstname']).drop(['lastname','firstname'],axis=1)
facts = facts[['pID','aID','depID','dateID','yID','points']]

In [167]:
publications.to_csv('Data/230122_publications.csv',index=False)
authors.to_csv('Data/230122_authors.csv',index=False)
years.to_csv('Data/230122_years.csv',index=False)
dates.to_csv('Data/230122_dates.csv',index=False)
departments.to_csv('Data/230122_departments.csv',index=False)
facts.to_csv('Data/230122_facts.csv',index=False)